## 1. 학습 데이터의 월 판매량(target data) (0,20)범위로 clip

## 2. hyper parameter 지정
- 'metric': 'rmse', # 평가지표 = rmse
- 'num_leaves': 300, # reaf node 수
- 'learning_rate': 0.005, # 디폴트0.1, 작을수록 예측성능이 높아질수있지만 과적합가능성도 올라감
- 'feature_fraction': 0.8, # 디폴트 1,개별 트리를 학습할 때 무작위로 선택되는 피쳐 비율
- 'bagging_fraction': 0.8, # 디폴트 1, 학습데이터 샘플링 비율
- 'bagging_freq': 3, # 디폴트 0, 몇번 iteration할때마다 다시 bagging을 수행할 지
- 'force_col_wise': True, # 디폴트 false, 컬럼수가 많을떄 메모리 비용을 줄일 수 있음
- 'random_state': 42 # seed 고정
- 'num_boost_round' : 1500 # 디폴트 100, 반복수행하는 횟수 
- 'early_stopping_rounds' : 150 # 디폴트 0, 검증결과가 150번동안 나아지지 않으면 멈춤
- 'categorical_feature' : ['shop_id', 'city_id', 'cat_id', 'main_cat', 'sub_cat'] # 범주형 피쳐 지정
- 'verbose_eval'=100 # 몇번째마다 평가지표 출력할지

## 3. 사용 데이터
- data_1 : 가장 기본 전처리(보고서 내 전처리 항목 참고)

## 4. submission 여부 : NO
- 검증 데이터에 대한 RMSE 감소가 없으므로 미진행

In [2]:
# 라이브러리 호출
import numpy as np
import pandas as pd
import time
from itertools import product 
import lightgbm

In [3]:
data = pd.read_pickle('./data/data_1.pkl')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8032887 entries, 827691 to 8860577
Data columns (total 29 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   month_id                    int8   
 1   shop_id                     int8   
 2   item_id                     int16  
 3   cnt_month                   float32
 4   city_id                     int8   
 5   cat_id                      int8   
 6   main_cat                    int8   
 7   sub_cat                     int8   
 8   year                        int16  
 9   month                       int8   
 10  sales_month                 int8   
 11  cnt_month_1                 float32
 12  cnt_month_2                 float32
 13  cnt_month_3                 float32
 14  item_id_mean_cnt_1          float32
 15  item_id_mean_cnt_2          float32
 16  item_id_mean_cnt_3          float32
 17  sub_cat_mean_cnt_1          float32
 18  sub_cat_mean_cnt_2          float32
 19  sub_cat_mean_cnt

In [4]:
data.isnull().sum()

month_id                           0
shop_id                            0
item_id                            0
cnt_month                     214200
city_id                            0
cat_id                             0
main_cat                           0
sub_cat                            0
year                               0
month                              0
sales_month                        0
cnt_month_1                        0
cnt_month_2                        0
cnt_month_3                        0
item_id_mean_cnt_1                 0
item_id_mean_cnt_2                 0
item_id_mean_cnt_3                 0
sub_cat_mean_cnt_1                 0
sub_cat_mean_cnt_2                 0
sub_cat_mean_cnt_3                 0
price_mean_mean_cnt_1              0
price_mean_mean_cnt_2              0
price_mean_mean_cnt_3              0
item_id/city_id_mean_cnt_1         0
item_id/city_id_mean_cnt_2         0
item_id/city_id_mean_cnt_3         0
shop_id/cat_id_mean_cnt_1          0
s

In [4]:
# train/validation/test data split
X_train = data[data['month_id'] < 33].drop(['cnt_month'], axis=1)
y_train = data[data['month_id'] < 33]['cnt_month'].clip(0,20)

X_valid = data[data['month_id'] == 33].drop(['cnt_month'], axis=1)
y_valid = data[data['month_id'] == 33]['cnt_month'].clip(0,20)

X_test = data[data['month_id'] == 34].drop(['cnt_month'], axis=1)

In [7]:
# hyper parameter 지정
params = {'metric': 'rmse',
          'num_leaves': 300, 
          'learning_rate': 0.005, 
          'feature_fraction': 0.8, 
          'bagging_fraction': 0.8,
          'bagging_freq': 3,
          'force_col_wise': True, 
          'random_state': 42}

cat = ['shop_id', 'city_id', 'cat_id', 'main_cat', 'sub_cat']

# 데이터 셋 지정
train = lightgbm.Dataset(X_train, y_train)
valid = lightgbm.Dataset(X_valid, y_valid)
 
# 모델 훈련
start = time.time()
lgb_model = lightgbm.train(params=params,
                      train_set=train,
                      num_boost_round=1500, 
                      valid_sets=(train, valid), 
                      early_stopping_rounds=150,
                      categorical_feature=cat, 
                      verbose_eval=50)
end = time.time()
print(f'소요시간(s) : {end-start}')

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_id', 'city_id', 'main_cat', 'shop_id', 'sub_cat']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Total Bins 4423
[LightGBM] [Info] Number of data points in the train set: 7596885, number of used features: 28


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] Start training from score 0.306474
Training until validation scores don't improve for 150 rounds
[50]	training's rmse: 1.10537	valid_1's rmse: 0.979518
[100]	training's rmse: 1.01682	valid_1's rmse: 0.914986
[150]	training's rmse: 0.956046	valid_1's rmse: 0.874893
[200]	training's rmse: 0.913447	valid_1's rmse: 0.85101
[250]	training's rmse: 0.883342	valid_1's rmse: 0.836754
[300]	training's rmse: 0.861518	valid_1's rmse: 0.829909
[350]	training's rmse: 0.845368	valid_1's rmse: 0.82568
[400]	training's rmse: 0.832717	valid_1's rmse: 0.825107
[450]	training's rmse: 0.822385	valid_1's rmse: 0.825251
[500]	training's rmse: 0.814209	valid_1's rmse: 0.825225
[550]	training's rmse: 0.807315	valid_1's rmse: 0.826481
Early stopping, best iteration is:
[429]	training's rmse: 0.826561	valid_1's rmse: 0.824776
소요시간(s) : 382.3573741912842
